In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from scipy.stats import pearsonr



cnn_model = load_model("/Users/samarpanpoudel/Desktop/model 1/saved_models/model1.h5")
resnet_model = load_model("/Users/samarpanpoudel/Desktop/Transfer Learning/saved_models/resnet1.h5")



folder_path = r"/Users/samarpanpoudel/Desktop/test"

image_files = sorted(os.listdir(folder_path))[:12]

cnn_predictions = []
resnet_predictions = []

for img_name in image_files:
    
    img_path = os.path.join(folder_path, img_name)
    
    # Load image (IMPORTANT: 256x256)
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    #img_array1 = img_array / 255.0   # same preprocessing as training
    img_array = np.expand_dims(img_array, axis=0)
    
    # CNN prediction
    cnn_pred = cnn_model.predict(img_array, verbose=0)
    cnn_prob_class1 = float(cnn_pred[0][1])
    cnn_predictions.append(cnn_prob_class1)
    
    # ResNet prediction
    resnet_pred = resnet_model.predict(img_array, verbose=0)
    resnet_prob_class1 = float(resnet_pred[0][1])
    resnet_predictions.append(resnet_prob_class1)

# Convert to numpy
cnn_predictions = np.array(cnn_predictions)
resnet_predictions = np.array(resnet_predictions)



correlation, p_value = pearsonr(cnn_predictions, resnet_predictions)

print("CNN class-1 probabilities:\n", cnn_predictions)
print("\nResNet class-1 probabilities:\n", resnet_predictions)
print("\nPearson Correlation:", correlation)
print("P-value:", p_value)

CNN class-1 probabilities:
 [0.01784083 0.01872507 0.01874232 0.03732447 0.04102376 0.32590681
 0.99975103 0.99968445 0.99974281 0.99879646 0.80302519 0.99986565]

ResNet class-1 probabilities:
 [1.00685339e-02 1.23713585e-02 6.73966780e-02 2.14240327e-03
 1.19739363e-03 8.22069123e-05 1.00000000e+00 9.99986887e-01
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]

Pearson Correlation: 0.9756302170012292
P-value: 6.498110820144197e-08


In [34]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

folder_path = r"/Users/samarpanpoudel/Desktop/test"
image_files = sorted(os.listdir(folder_path))[:12]

img_list = []

for img_name in image_files:
    img_path = os.path.join(folder_path, img_name)
    
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    #img_array = img_array / 255.0   # same preprocessing as training
    
    img_list.append(img_array)

# Convert to batch
img_array_batch = np.array(img_list)

print("Batch shape:", img_array_batch.shape)

Batch shape: (12, 256, 256, 3)


In [35]:
cnn_preds = cnn_model.predict(img_array_batch, verbose=0)
resnet_preds = resnet_model.predict(img_array_batch, verbose=0)

print("CNN predicted classes:")
print(np.argmax(cnn_preds, axis=1))

print("ResNet predicted classes:")
print(np.argmax(resnet_preds, axis=1))

CNN predicted classes:
[0 0 0 0 0 0 1 1 1 1 1 1]
ResNet predicted classes:
[0 0 0 0 0 0 1 1 1 1 1 1]


In [36]:
print("CNN raw predictions:")
print(cnn_preds)

print("\nResNet raw predictions:")
print(resnet_preds)

CNN raw predictions:
[[9.8215926e-01 1.7840810e-02]
 [9.8127490e-01 1.8725082e-02]
 [9.8125774e-01 1.8742297e-02]
 [9.6267551e-01 3.7324488e-02]
 [9.5897627e-01 4.1023765e-02]
 [6.7409283e-01 3.2590723e-01]
 [2.4897937e-04 9.9975109e-01]
 [3.1552021e-04 9.9968445e-01]
 [2.5716724e-04 9.9974287e-01]
 [1.2035101e-03 9.9879652e-01]
 [1.9697466e-01 8.0302536e-01]
 [1.3437607e-04 9.9986565e-01]]

ResNet raw predictions:
[[9.8993146e-01 1.0068477e-02]
 [9.8762858e-01 1.2371452e-02]
 [9.3260384e-01 6.7396209e-02]
 [9.9785757e-01 2.1424193e-03]
 [9.9880260e-01 1.1973982e-03]
 [9.9991781e-01 8.2205028e-05]
 [9.2106553e-09 1.0000000e+00]
 [1.3102532e-05 9.9998689e-01]
 [7.7862788e-10 1.0000000e+00]
 [2.2540569e-17 1.0000000e+00]
 [1.7887380e-08 1.0000000e+00]
 [3.3675289e-09 1.0000000e+00]]
